In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import glob

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import numpy as np
import itertools
import shutil
import matplotlib.pyplot as plt
from numpy.random import seed
np.random.seed(101)
tf.random.set_seed(101)

print(tf.__version__)

from tensorflow.keras.layers import Dropout, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from keras.applications.mobilenet import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


2.11.0


In [ ]:
train_path = 'gdrive/My Drive/CV_Project/train'
valid_path = 'gdrive/My Drive/CV_Project/test'

num_train_samples = 800
num_val_samples = 160
train_batch_size = 10
val_batch_size = 10
image_size = 224

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

print(int(train_steps))
print(int(val_steps))


80
16


In [ ]:
datagen = ImageDataGenerator(
    preprocessing_function= preprocess_input)

train_batches = datagen.flow_from_directory(train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=train_batch_size)

valid_batches = datagen.flow_from_directory(train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=val_batch_size)

# Note: shuffle=False causes the test dataset to not be shuffled
test_batches = datagen.flow_from_directory(valid_path,
                                            target_size=(image_size,image_size),
                                            batch_size=1,
                                            shuffle=False)

FileNotFoundError: ignored

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNet
from efficientnet.tfkeras import EfficientNetB0

# Create an ImageDataGenerator for preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
     validation_split=0.2,
)

# Load and preprocess training images
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Load and preprocess validation images
validation_generator = datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Load the MobileNet model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add a custom output layer for classification
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(4, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# Evaluate the model on the test set
test_generator = datagen.flow_from_directory(
    valid_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

metrics = model.evaluate_generator(test_generator)
print('Test accuracy:', metrics[1])


Found 640 images belonging to 4 classes.
Found 160 images belonging to 4 classes.
17225924/17225924 [==============================] - 1s 0us/step


<ipython-input-7-332fbbe585be>:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
20/20 [==============================] - 263s 14s/step - loss: 9.0969 - accuracy: 0.4516 - val_loss: 1.0545 - val_accuracy: 0.5562
Epoch 2/10
20/20 [==============================] - 42s 2s/step - loss: 0.8171 - accuracy: 0.7078 - val_loss: 1.0188 - val_accuracy: 0.6000
Epoch 3/10
20/20 [==============================] - 42s 2s/step - loss: 0.5575 - accuracy: 0.7641 - val_loss: 0.9352 - val_accuracy: 0.6562
Epoch 4/10
20/20 [==============================] - 52s 3s/step - loss: 0.4461 - accuracy: 0.8047 - val_loss: 1.1010 - val_accuracy: 0.6125
Epoch 5/10
20/20 [==============================] - 42s 2s/step - loss: 0.2833 - accuracy: 0.8875 - val_loss: 1.0034 - val_accuracy: 0.6938
Epoch 6/10
20/20 [==============================] - 44s 2s/step - loss: 0.1184 - accuracy: 0.9688 - val_loss: 1.1322 - val_accuracy: 0.7063
Epoch 7/10
20/20 [==============================] - 44s 2s/step - loss: 0.0466 - accuracy: 0.9906 - val_loss: 1.0830 - val_accuracy: 0.6812
Epoch 8/10
20/20 [

<ipython-input-7-332fbbe585be>:67: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  metrics = model.evaluate_generator(test_generator)


Test accuracy: 0.956250011920929


In [ ]:
!pip install efficientnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 KB 6.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing.image import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = load_img(path, target_size=(224, 224))
  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  temp=0
  x=0
  c=0
  for i in classes[0]:
    if(i>temp):
     
      temp=i
      x=c
    c+=1
  class_names={}
  class_names={0:'Chives',1:'Oregano',2:'Parsley',3:'Thyme'}
  print(class_names[x])

Saving commodity-chives.jpg to commodity-chives.jpg
1/1 [==============================] - 1s 675ms/step
Chives


In [ ]:
temp

In [ ]:
classes

In [ ]:
print(train_batches.class_indices)
